# Control Chart Maker Template (Individual Observations)
by Nathan Whitehead

original code source: https://towardsdatascience.com/quality-control-charts-guide-for-python-9bb1c859c051

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics as st

# import data
x=pd.read_csv('yourdata.csv', index_col=0) #import your data here, remove index_col if it does not contain an index column

#import constants
constants = pd.read_csv('control chart constants.csv', index_col=0)
#perform log base 10 transform on the data
x['log10'] = np.log10(x['uniformity'])
x['shifted'] = x['log10'].shift(1)
x['MR'] = x['log10']-x['shifted']

x.to_csv('xcsv.csv')
#constants
n=2 #n is 2 for moving range
D_3 = constants.at[n,'D_3']
D_4 = constants.at[n,'D_4']
d_2 = constants.at[n,'d_2']


#initializing all variables to simplify future code
MR_bar = np.nanmean(x['MR']) #must use nanmean because of NaN value in first row of MR.
MR_UCL = MR_bar*D_4
MR_LCL = MR_bar*D_3
x_bar = st.mean(x['log10'])
x_UCL = x_bar + 3*(MR_bar/d_2)
x_LCL = x_bar - 3*(MR_bar/d_2)


# Output Centerline and Control Limits

In [ ]:
print("-----MR chart-----")
print("MR-bar:",MR_bar)    
print("UCL:",MR_UCL)
print("LCL:",MR_LCL)

print("-----x chart-----")
print("x-bar:",x_bar)    
print("UCL:",x_UCL)
print("LCL:",x_LCL)

# Plot Control Chart

In [ ]:
# Plot charts
fig, axs = plt.subplots(2, figsize=(10,7))

# MR chart
axs[0].plot(x['MR'], linestyle='-', marker='o', color='black')
axs[0].axhline(MR_UCL, color='red', linestyle='dashed')
axs[0].axhline(MR_LCL, color='red', linestyle='dashed')
axs[0].axhline(MR_bar, color='blue')
axs[0].set_title('MR Chart')
axs[0].set(xlabel='Observation', ylabel='Moving Range')

#X chart
axs[1].plot(x['log10'], linestyle='-', marker='o', color='black')
axs[1].axhline(x_UCL, color='red', linestyle='dashed')
axs[1].axhline(x_LCL, color='red', linestyle='dashed')
axs[1].axhline(x_bar, color='blue')
axs[1].set_title('x Chart')
axs[1].set(xlabel='Group', ylabel='log(uniformity)')
fig.tight_layout()

# Check for out of control points

In [ ]:
# Validate points out of control limits for R chart
i = 0
control = True
for group in x['MR']:
    if group > MR_UCL or group < MR_LCL:
        print('Group', i, 'out of range cotrol limits!')
        control = False
    i += 1
i=0
for group in x['log10']:
    if group > x_UCL or group < x_LCL:
        print('Group', i, 'out of range cotrol limits!')
        control = False
    i += 1
if control == True:
    print('p chart: All points within control limits.')